# 2. 온라인 선호지수 개발

## 2.2 협업 필터링을 통한 선호도 점수 행렬 도출

In [1]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [2]:
#데이터가져오기(기본 데이터)
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#데이터가져오기(created from 'Developing Online Preference 1')
Client_Behavior = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF2/Client_Behavior.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#첫번째 column 모두 삭제
Client_Behavior = Client_Behavior.drop(Client_Behavior.columns[[0]], axis=1)

print(Client_Behavior.columns.values[0])

CLNT_ID


### 2) 검색횟수를 이용한 온라인 선호지수 생성

In [ ]:
Search_CNT = pd.read_csv("C:/Users/lsh93/Desktop/data/UBCF2/Search_CNT.csv")
Search_CNT.head()

In [ ]:
Search_CNT = Search_CNT.rename(columns = {'Unnamed: 0':'CLNT_ID'})
Search_CNT.head()

In [ ]:
#Index 추출
row_names = Search_CNT['CLNT_ID'].as_matrix()
row_names

In [ ]:
#Columns 추출 
col_names = Search_CNT.colums.values
col_names = col_names[1:] # 왜 이렇게 한건지 확인하기
col_names 

In [ ]:
PC = Search_Count.as_matrix()
PC = PC[:,1:]
print(PC.shape)
print(PC) 

In [ ]:
################
Search_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
Search_Count.head()

In [ ]:
print(Search_Count.shape)

In [ ]:
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#test set과 training set으로 나누기
#Array 의 형태로 나누어주는 듯 함
PC_train, PC_test = train_test_split(Search_Count, test_size = 0.33, random_state = 50)

In [ ]:
print(PC_train)
print(PC_train.shape)
print(PC_test)
print(PC_test.shape)

In [ ]:
#DataFrame 형태로 바꾸기
PC_train = pd.DataFrame(PC_train)
PC_train.head()

In [ ]:
#DataFrame 형태로 바꾸기
PC_test = pd.DataFrame(PC_test)
PC_train.head()

### Cosine Distance 를 이용한 선호점수 계산

In [ ]:
#row 길이가 너무 커서 줄일 필요가 있음 (나중에 클러스터링 나오면 이 과정은 생략해도 됨)
#DataFrame 형태
train = PC_train[0:1000]
test = PC_test[0:330]

In [ ]:
#Array 형태로 변환 
train_mat = train.as_matrix()
test_mat = test.as_matrix()
print(train_mat.shape)
print(test_mat.shape)

#### Cosine Similarity 계산

In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
#MSE 계산 함수 작성
from sklearn.metrics import mean_squared_error
def get_mse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

In [ ]:
dist_out = 1 - pairwise_distances(train_mat, metric="cosine")
dist_out

In [ ]:
dist_out.shape

#### w/o k means clustering

In [ ]:
#Client의 알려지지 않은 평가 예측 
#거리행렬과 평가행렬의 내적을 구하고 평가수의 데이터를 정규화해 계산
user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T

In [ ]:
#모델의 예측오차율
get_mse(user_pred, train_mat)

In [ ]:
#테스트 데이터를 사용해 정확도를 확인해보기
get_mse(user_pred, test_mat)

#### w/ k means clustering

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
k = 5
neigh = NearestNeighbors(k, 'cosine')

In [ ]:
neigh.fit(train_mat)

In [ ]:
#각 사용자별로 가장 유사한 5명의 사용자 찾고 유사도 계산
top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)

In [ ]:
print(top_k_distances.shape)
print(top_k_users.shape)

In [ ]:
top_k_users[4]

In [ ]:
#훈련 데이터의 알려지지 않은 구매 예측
user_pred_k = np.zeros(train_mat.shape)

In [ ]:
for i in range(temp_train.shape[0]):
    user_pred_k[i,:] = top_k_distances[i].T.dot(temp_train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T

In [ ]:
user_pred_k.shape

In [ ]:
user_pred_k

In [ ]:
##### nan값이 있어서 mse를 계산할 수 없음
online_pref = pd.DataFrame(user_pred_k)
online_pref

In [ ]:
#nan가 들어간 element의 row 인덱스 반환
na_element_index = np.where(np.isnan(online_pref))
na_element_index
#row인덱스가 중복해서 나옴

In [ ]:
#nan가 들어간 row의 인덱스 반환
na_row_index = np.unique(na_element_index[0])
na_row_index

In [ ]:
#nan가 들어간 row를 삭제한다 -> 그에 해당하는 client 를 삭제한다
#제거할 client id의 목록
removed_CLNT_ID = temp_train.index.values[na_row_index]
removed_CLNT_ID.shape

In [ ]:
#제거하지 않고 남을 client id의 목록
existed_CLNT_ID = np.delete(temp_train.index.values,na_row_index)
existed_CLNT_ID.shape

#### nan 포함 row 제거

In [ ]:
online_pref2 = online_pref.dropna()
online_pref2.shape

In [ ]:
user_pred_k2 = online_pref2.as_matrix()
user_pred_k2.shape

In [ ]:
user_pred_k2

In [ ]:
train_mat2 = train_mat
exist_idx = np.delete([range(1000)],na_row_index)
train_mat2 = train_mat2[exist_idx,:]
train_mat2.shape

In [ ]:
#모델의 개선 여부 확인 -> 개선됨! 
get_mse(user_pred_k2, train_mat2)

In [ ]:
get_mse(user_pred_k2, temp_test_mat)

- 검색횟수 + UBCF 기반 선호점수 matrix 생성 (index, column 모두 달아서)

In [ ]:
col_names = Search_Count.columns.values
row_names = existed_CLNT_ID
print(col_names.shape)
print(row_names.shape)
print(user_pred_k2.shape)

In [ ]:
Online_Preference_Search = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)
Online_Preference_Search.head()

In [ ]:
Online_Preference_Search.shape

In [ ]:
#데이터 내보내기
Online_Preference_Search.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Online_Preference_Search.csv", encoding="UTF-8")